In [38]:
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import OpenAIWhisperParser
from langchain_community.document_loaders.parsers.audio import OpenAIWhisperParserLocal
from typing import List
from langchain.schema import Document

In [39]:
urls = ["https://www.youtube.com/watch?v=FgzM3zpZ55o",
        "https://www.youtube.com/watch?v=E3f2Camj0Is",
        "https://www.youtube.com/watch?v=dRIhrn8cc9w",
        "https://www.youtube.com/watch?v=j080VBVGkfQ",
        "https://www.youtube.com/watch?v=buptHUzDKcE",
        "https://www.youtube.com/watch?v=gOV8-bC1_KU",
        "https://www.youtube.com/watch?v=V7CY68zH6ps",
        "https://www.youtube.com/watch?v=8LEuyYXGQjU",
        "https://www.youtube.com/watch?v=E-_ecpD5PkE",
        "https://www.youtube.com/watch?v=o_i5F1zGPLs",
        "https://www.youtube.com/watch?v=RN8qpSs8ozY",
        "https://www.youtube.com/watch?v=jJ7JbQBTChM",
        "https://www.youtube.com/watch?v=Hg_uyWezMM0",
        "https://www.youtube.com/watch?v=zPU1SRHuAW8",
        "https://www.youtube.com/watch?v=vDF1BYWhqL8"]

save_dir = "../../data/audio/youtube/"

loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParserLocal(lang_model="openai/whisper-tiny"))

docs = loader.load()


Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[youtube] Extracting URL: https://www.youtube.com/watch?v=FgzM3zpZ55o
[youtube] FgzM3zpZ55o: Downloading webpage
[youtube] FgzM3zpZ55o: Downloading ios player API JSON
[youtube] FgzM3zpZ55o: Downloading m3u8 information
[info] FgzM3zpZ55o: Downloading 1 format(s): 140
[download] Destination: ../../data/audio/youtube//Stanford CS234： Reinforcement Learning ｜ Winter 2019 ｜ Lecture 1 - Introduction - Emma Brunskill.m4a
[download]  49.5% of   61.01MiB at   19.51MiB/s ETA 00:01  

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x707e81b87370>>
Traceback (most recent call last):
  File "/home/matthieu/UbuntuData/PycharmProjects/snoop/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [40]:
print(type(docs))

# Print keys of docs
print(docs[10].page_content)

<class 'list'>
 Hi everybody, I'm Emma Brentskill. I'm an Assistant Professor in Computer Science and welcome to CS 234, which is reinforcement learning class, which is designed to be sort of an entry level master's or PhD student in an introduction to reinforcement learning. So what we're going to do today is I'm going to start with just a really short brief overview of what is reinforcement learning. And then we're going to go through course logistics. And when I go through course logistics, I'll also pause and ask for any questions about logistics. The website is now live and so that's also the best source of information about the class. That MPAZ will be the best source of information. So I'll stop there. We'll be get to that part to ask if there's anything that don't go over that you have questions about. And if you have questions about the wait list or any particular things relating to your own circumstance feel free to come up to me at the end. And then the third part of the cla

In [ ]:
for i, doc in enumerate(docs):
    print(f"Document {i}: {doc.metadata['source']}")

In [ ]:
docs_sorted = sorted(docs, key=lambda x: x.metadata["source"])

In [ ]:
docs_sorted[0]

In [ ]:
for i, doc in enumerate(docs_sorted):
    print(f"Document {i}: {doc.metadata['source']}")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def split_documents(documents: List[Document]):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=100,
    )
    chunks = splitter.split_documents(documents)
    print(f"Number of chunks: {len(chunks)}, for {len(documents)} documents")
    print(f"Split documents done !")
    print(type(chunks))
    return chunks

In [ ]:
chunks = split_documents(docs)

In [ ]:
chunks[99]

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)

In [ ]:
# Split them
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_text(text)

In [ ]:
# Build an index
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_texts(splits, embeddings)

In [ ]:
# Build a QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)

In [ ]:
# Ask a question!
query = "What are the key points to make the AI good decision ?"
qa_chain.invoke(query)